In [ ]:
!pip install svetlanna
!pip install reservoirpy matplotlib tqdm requests av scikit-image py-cpuinfo gputil pandas

In [1]:
from svetlanna.axes_math import tensor_dot, cast_tensor
from svetlanna.wavefront import mul
import svetlanna as sv
import torch

В данном примере показан пример работы с тензорными осями.

Оси представляют собой имена последних осей тензора.
Оси хранятся в кортежах строк.
Например, тензор с формой `(..., N, M, L)` может иметь кортеж осей `(a, b, c)`, где измерение `a` имеет `N` точек, `b` имеет `M` точек, а `c` имеет `C` точек.

In [2]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
a_axes = ('a',)
a.shape

torch.Size([2, 3])

Любой тензор можно перенести на другие оси, так что форма тензора будет изменена.

In [3]:
new_axes = ('b', 'a', 'c')
a_casted = cast_tensor(a, a_axes, new_axes)

a_casted.shape  # new axes was added to the second position and the end of the tensor

torch.Size([2, 1, 3, 1])

Чтобы выполнить скалярное произведение тензора, можно использовать функцию tensor_dot.
Например, если тензор `A` имеет оси `(a, b, c)`, а тензор `B` имеет оси `(b,)`, результат произведения будет иметь оси `(a, b, c)`.
Формула такого произведения следующая:
$$
A \cdot B = \sum_{i_a, i_b, i_c} (A_{i_a, i_b, i_c} \cdot B_{i_b}) \vec{e_{i_a}}\vec{e_{i_b}}\vec{e_{i_c}}
$$
Другими словами, каждое тензорное представление `A[:,i,:]` было умножено на `B[i]`.

Основное правило следующее: совпадающие оси будут умножаться.

In [4]:
A = torch.tensor([[[1, 2], [3, 4]]])
A_axes = ('a', 'b', 'c')

B = torch.tensor([10, 20])
B_axes = ('b', )

C, new_axes = tensor_dot(A, B, A_axes, B_axes)
C, new_axes

(tensor([[[10, 20],
          [60, 80]]]),
 ('a', 'b', 'c'))

В случае, когда `B` имеет ось, не представленную в осях `A`, результирующие тензорные оси будут объединением осей `B` и осей `A`:

In [5]:
A = torch.tensor([[1, 2], [3, 4]])
A_axes = ('a', 'b')

B = torch.tensor([10, 20, 30])
B_axes = ('c', )

C, new_axes = tensor_dot(A, B, A_axes, B_axes)
C, new_axes

(tensor([[[ 10,  20,  30],
          [ 20,  40,  60]],
 
         [[ 30,  60,  90],
          [ 40,  80, 120]]]),
 ('a', 'b', 'c'))

Полученные оси тензора можно проверить, совпадают ли они с осями тензора `A` или нет, используя аргумент `preserve_a_axis`.
Если `preserve_a_axis=True`, предыдущий пример завершится ошибкой с вызовом AssertionError.

Давайте определим параметры моделирования

In [6]:
Nx = 50
Ny = 100
Nwl = 4

sim_params = sv.SimulationParameters(
    {
        'W': torch.linspace(-10, 10, Nx),
        'H': torch.linspace(-10, 10, Ny),
        'wavelength': torch.linspace(1, 5, Nwl),
    }
)

# можно посмотреть последовательность осей, которая используется во время моделирования
sim_params.axes.names

('wavelength', 'H', 'W')

Предположим, что во время моделирования необходимо выполнить умножение волнового фронта и некоторой функции передачи `T`, которая определена на сетке (x,y)

In [7]:
wavefront = sv.Wavefront(torch.rand(sim_params.axes_size(sim_params.axes.names)))

T = torch.rand(Ny, Nx)
T_axis = ('H', 'W')

c1, c1_axis = tensor_dot(
    wavefront,
    T,
    sim_params.axes.names,
    T_axis
)
c1.shape

torch.Size([4, 100, 50])

Для вычисления `c1` можно использовать сокращение:

In [8]:
c2 = mul(
    wavefront,
    T,
    T_axis,
    sim_params
)

assert torch.allclose(c1, c2)

Функцию `mul` можно использовать в методе `forward` элемента.

Продемонстрированный подход позволяет не думать об осях волнового фронта, предоставленных пользователем во время моделирования, и позволяет пользователю добавлять собственные оси волнового фронта в любом порядке:

In [9]:
sim_params2 = sv.SimulationParameters(
    {
        'wavelength': torch.linspace(1, 5, Nwl),
        'H': torch.linspace(-10, 10, Ny),
        'polarization': torch.tensor([0, 1]),
        'W': torch.linspace(-10, 10, Nx),
    }
)

sim_params2.axes.names

('W', 'polarization', 'H', 'wavelength')

In [10]:
# new wavefront
new_wavefront = sv.Wavefront(torch.rand(sim_params2.axes_size(sim_params2.axes.names)))
new_wavefront.shape

torch.Size([50, 2, 100, 4])

Предыдущий код по-прежнему работает только с изменением аргументов волнового фронта и параметров моделирования:

In [11]:
c3 = mul(
    new_wavefront,
    T,
    T_axis,
    sim_params2
)